In [ ]:
!pip install pandas

In [1]:
import pandas as pd
from collections import Counter
import re
import numpy as np

In [2]:
df = pd.read_csv('treatment.csv')

In [3]:
treatmentList = df['treatmentstring'].tolist()
treatmentList2 = []
for treatment in treatmentList:
    for treat in treatment.split('|'):
        treatmentList2.append(treat)

In [4]:
dfTreatment = df[df['treatmentstring'].str.contains("cardiovascular")]
dfTreatment = dfTreatment[['patientunitstayid','treatmentstring','activeupondischarge']]

In [5]:
listPatients = dfTreatment['patientunitstayid'].tolist()
repetitionsPatient = Counter(listPatients)

In [6]:
dfDiagnosis = pd.read_csv('diagnosis.csv', na_values="nan")
dfDiagnosis = dfDiagnosis.dropna()

In [7]:
listICDcodes = dfDiagnosis['icd9code'].tolist()
validIndices = []
i=0
for icd in listICDcodes:
    icdCode1 = icd.split(',')[0]
    if re.match(r'^-?\d+(?:\.\d+)$', icdCode1) is not None and float(icdCode1) >= 390 and float(icdCode1) <= 459:
        validIndices.append(i)
    i+=1

dfDiagnosisFiltered = dfDiagnosis.iloc[validIndices]
filteredPatientIds = set(dfDiagnosisFiltered['patientunitstayid'].tolist())

False

In [8]:
dfDiagnosis = pd.read_csv('diagnosis.csv', na_values="nan")
dfDiagnosis = dfDiagnosis.dropna()

In [9]:
dfVitalAperiodic = pd.read_csv('vitalAperiodic.csv')
dfVitalAperiodic = dfVitalAperiodic.groupby(['patientunitstayid'], as_index=False).mean()[['patientunitstayid','vitalaperiodicid', 'observationoffset','noninvasivesystolic',
                                                                                           'noninvasivediastolic','noninvasivemean']]

,patientunitstayid,vitalaperiodicid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean
0,141764,3661420.5,204.125000,172.750000,93.375000,125.625000
1,141765,5600897.5,961.900000,146.033333,72.800000,98.400000
2,143870,5022024.5,296.500000,102.500000,53.166667,74.500000
3,144815,1196461.5,480.333333,114.388889,72.777778,88.777778
4,145427,2150246.5,358.250000,116.666667,62.833333,83.208333
...,...,...,...,...,...,...
2326,3351763,443446038.5,2876.161765,135.985294,63.323529,89.279412
2327,3352230,443136851.0,2360.507246,117.101449,65.260870,84.304348
2328,3352231,445040552.0,655.600000,128.000000,71.800000,91.800000
2329,3352333,442861118.5,1505.014286,146.600000,81.628571,106.228571


In [10]:
dfPatient = pd.read_csv('patient.csv')
dfPatientNew = dfPatient[['patientunitstayid', 'gender', 'age', 'admissionweight', 'admissionheight', 'ethnicity']]

In [11]:
dfPatientNew = dfPatientNew.merge(dfVitalAperiodic, how='left', left_on='patientunitstayid', right_on='patientunitstayid')

In [12]:
dfPatientNew['cardioDiagnosis'] = dfPatientNew['patientunitstayid'].apply(lambda x : x in filteredPatientIds)

In [13]:
dfTreatment = pd.read_csv('treatment.csv')[['patientunitstayid','treatmentstring']] 

In [14]:
dfTreatment = dfTreatment.groupby(['patientunitstayid'], as_index=False).agg({'treatmentstring': ' '.join})
dfTreatment['treated'] = np.where(dfTreatment['treatmentstring'].str.contains("cardiovascular"), 'True', 'False')
dfTreatment = dfTreatment[['patientunitstayid', 'treated']]

In [15]:
dfPatientNew = dfPatientNew.merge(dfTreatment, how='left', left_on='patientunitstayid', right_on='patientunitstayid')
dfPatientNew = dfPatientNew.dropna()
dfPatientNew

In [16]:
dfPatientNew.to_csv('patientData.csv', index=False)